# 5. Run the Batch AI job from Docker
This notebook will walk through the steps to create a Docker image, test it locally, and upload it to Dockerhub to be consumed later in the tutorial when we operaitonalize the workflow.

This notebook assumes a basic understanding of Docker and Dockerfiles and how to use the Docker commandline to create an Docker image. To learn more about Docker, see [here.](https://docs.docker.com/get-started/)

---

First we need to set up some variables for Docker:
- `docker_image_tag` - this is the tag that you wish to name your Docker image
- `docker_user` - your Dockerhub username

In [ ]:
docker_image_tag = "<docker-image-tag>"
docker_user = "<docker-username>"

Save these variables to the .env file

In [ ]:
!dotenv set DOCKER_IMAGE_TAG $docker_image_tag
!dotenv set DOCKER_USER $docker_user

Load the .env file so you can access these variables as environment variables.

In [ ]:
%load_ext dotenv
%dotenv

Create a requirements.txt file that will be used in the docker image. Because the Docker image is only responsible for running the `create_job.py` script, we only need to include the packages that that script requires.

In [ ]:
%%weritefile az/requirements.txt
azure==4.0.0
azure-mgmt-batchai==2.0.0
iteration-utilities==0.7.0

Write the Dockerfile that will be used to create the Docker image.

In [ ]:
%%writefile az/Dockerfile
FROM python:3.6-jessie
WORKDIR /job
ADD create_job.py /job
ADD util /job/util
ADD requirements.txt /job

RUN pip install --trusted-host pypi.python.org -r requirements.txt

CMD ["python", "create_job.py"]

Build the Dockerfile.

In [ ]:
! cd az && sudo docker build -t $DOCKER_IMAGE_TAG .

Now lets test that the image worked as expected. Run the image in the background (detached) and pass it the environment variables needed to run the `create_job.py` script.

In [ ]:
%%bash
sudo docker run -d --name $DOCKER_IMAGE_TAG \
    -e AAD_CLIENT_ID=$AAD_CLIENT_ID \
    -e AAD_SECRET=$AAD_SECRET \
    -e AAD_TENANT=$AAD_TENANT \
    -e STORAGE_ACCOUNT_NAME=$STORAGE_ACCOUNT_NAME \
    -e STORAGE_ACCOUNT_KEY=$STORAGE_ACCOUNT_KEY \
    -e ADMIN_USER_NAME=$ADMIN_USER_NAME\
    -e ADMIN_USER_PASSWORD=$ADMIN_USER_PASSWORD \
    -e SUBSCRIPTION_ID=$SUBSCRIPTION_ID \
    -e RESOURCE_GROUP=$RESOURCE_GROUP \
    -e REGION=$REGION \
    -e AZURE_CONTAINER_NAME=$AZURE_CONTAINER_NAME \
    -e FS_SCRIPT=$FS_SCRIPT \
    -e FS_STYLE_IMG=$FS_STYLE_IMG \
    -e FS_CONTENT_DIR=$FS_CONTENT_DIR \
    -e FS_OUTPUT_DIR_PREFIX=$FS_OUTPUT_DIR_PREFIX \
    -e FS_LOGGER_DIR_PREFIX=$FS_LOGGER_DIR_PREFIX \
    -e WORKSPACE=$WORKSPACE \
    -e EXPERIMENT_PREFIX=$EXPERIMENT_PREFIX \
    -e JOB_NAME_PREFIX=$JOB_NAME_PREFIX \
    -e JOB_NODE_COUNT=$JOB_NODE_COUNT \
    -e JOB_BATCH_SIZE=$JOB_BATCH_SIZE \
    -e CLUSTER_NAME=$CLUSTER_NAME \
    -e CLUSTER_CONTAINER_MNT_PATH=$CLUSTER_CONTAINER_MNT_PATH \
    -e STYLE_WEIGHT=$STYLE_WEIGHT \
    -e CONTENT_WEIGHT=$CONTENT_WEIGHT \
    -e NUM_STEPS=$NUM_STEPS \
    -e IMAGE_SIZE=$IMAGE_SIZE \
    $DOCKER_IMAGE_TAG

Lets take a look at the logs of the running Docker container to see that the output looks as expected.

In [ ]:
!sudo docker logs $DOCKER_IMAGE_TAG

Now that we've testing the image locally, lets push it up to Dockerhub so that we can use it when operationalizing this workflow with Logic Apps.

In [ ]:
!sudo docker tag $DOCKER_IMAGE_TAG $DOCKER_USER/$DOCKER_IMAGE_TAG
!sudo docker push $DOCKER_USER/$DOCKER_IMAGE_TAG

---

## Conclusion
In this tutorial, we've setup our `create_job.py` script to run inside of a Docker image. We've also pushed the image that we built locally up to Dockerhub to be consumed later on in the tutorial.

Next, lets use this Docker image to operationalize this workflow. To do so, we need to [deploy Logic App and trigger the workflow.](06_deploy_and_test_logic_apps.ipynb)